# Test CNN dla NLP - analiza sekwencji słów

Ten notebook testuje CNN 1D na analizie prostych sekwencji "słów" (tokenów), podobnie jak w Flux do zadań NLP. Używamy bardzo małych danych, które można policzyć ręcznie.

## Importowanie modułów i bibliotek

In [1]:
include("../MyReverseDiff.jl")
include("../MyEmbedding.jl")
include("../MyMlp.jl")

using .MyReverseDiff
using .MyEmbedding
using .MyMlp
using Printf
using LinearAlgebra
using Random

# Ustaw seed dla reprodukowalności
Random.seed!(42)

println("Moduły załadowane pomyślnie!")

Moduły załadowane pomyślnie!


## 1. Przygotowanie danych tekstowych

Utworzymy mini-słownik i 2 proste "zdania" z różnymi wzorami sekwencyjnymi.

In [2]:
# Mini słownik (bardzo mały dla prostoty)
vocab = ["good", "bad", "movie", "very", "<PAD>"]
vocab_size = length(vocab)
embedding_dim = 3  # Małe embedding dla łatwych obliczeń
sequence_length = 4  # Długość sekwencji

println("Słownik:")
for (i, word) in enumerate(vocab)
    println("$i: $word")
end

println("\nParametry:")
println("- Rozmiar słownika: $vocab_size")
println("- Wymiar embedding: $embedding_dim")
println("- Długość sekwencji: $sequence_length")

Słownik:
1: good
2: bad
3: movie
4: very
5: <PAD>

Parametry:
- Rozmiar słownika: 5
- Wymiar embedding: 3
- Długość sekwencji: 4


In [3]:
# Sekwencje tekstowe jako indeksy
# Sekwencja 1: "very good movie <PAD>" → [5, 2, 4, 1] → pozytywna (klasa 1)
# Sekwencja 2: "very bad movie <PAD>" → [5, 3, 4, 1] → negatywna (klasa 0)

sequence_1 = Float32[4, 1, 3, 5]  # very good movie <PAD>
sequence_2 = Float32[4, 2, 3, 5]  # very bad movie <PAD>

# Konwertuj na format batch (sequence_length, batch_size)
X_batch = zeros(Float32, sequence_length, 2)
X_batch[:, 1] = sequence_1
X_batch[:, 2] = sequence_2

# Etykiety: sekwencja 1 → pozytywna (1), sekwencja 2 → negatywna (0)
y_batch = Float32[1.0 0.0]  # (1, 2) 

println("Sekwencja 1 (pozytywna): ", [vocab[Int(i)] for i in sequence_1])
println("Indeksy: ", sequence_1)
println("\nSekwencja 2 (negatywna): ", [vocab[Int(i)] for i in sequence_2])
println("Indeksy: ", sequence_2)
println("\nShape danych X: ", size(X_batch))
println("Etykiety y: ", y_batch)
println("Shape etykiet: ", size(y_batch))

Sekwencja 1 (pozytywna): ["very", "good", "movie", "<PAD>"]
Indeksy: Float32[4.0, 1.0, 3.0, 5.0]

Sekwencja 2 (negatywna): ["very", "bad", "movie", "<PAD>"]
Indeksy: Float32[4.0, 2.0, 3.0, 5.0]

Shape danych X: (4, 2)
Etykiety y: Float32[1.0 0.0]
Shape etykiet: (1, 2)


## 2. Definicja modelu CNN dla NLP

Architektura podobna do Flux:
- **Embedding**: słowa → wektory
- **Conv1D**: wykrywa n-gramy (wzory w sekwencjach)
- **MaxPool**: wybiera najważniejsze cechy
- **Flatten**: spłaszczenie
- **Dense**: klasyfikacja

In [4]:
# Parametry modelu
vocab_size = 5
embedding_dim = 3
conv_filters = 2        # Liczba filtrów CNN
kernel_size = 2         # Rozmiar kernela (analizuje 2 kolejne słowa)
hidden_size = 4         # Rozmiar warstwy ukrytej
output_size = 1         # Klasyfikacja binarna
batch_size = 2

println("Parametry modelu NLP:")
println("- Embedding: $vocab_size słów → $embedding_dim wymiarów")
println("- CNN: $conv_filters filtrów, kernel size $kernel_size (analizuje $kernel_size słowa naraz)")
println("- Warstwa ukryta: $hidden_size neuronów")
println("- Wyjście: $output_size neuron (sentiment pozytywny/negatywny)")
println("- Batch size: $batch_size")

Parametry modelu NLP:
- Embedding: 5 słów → 3 wymiarów
- CNN: 2 filtrów, kernel size 2 (analizuje 2 słowa naraz)
- Warstwa ukryta: 4 neuronów
- Wyjście: 1 neuron (sentiment pozytywny/negatywny)
- Batch size: 2


In [5]:
# Model: Embedding → Conv1D → Pool → Flatten → Dense → Dense
model = Chain(
    Embedding(vocab_size, embedding_dim; name="embedding"),
    ConvolutionBlock(conv_filters, kernel_size; name="conv1d"),
    PoolingBlock(3; name="maxpool"),  # Max pooling
    FlattenBlock(name="flatten"),
    Dense(8, hidden_size, relu; name="hidden"),
    Dense(hidden_size, output_size, σ; name="output")
)

println("Model CNN dla NLP utworzony pomyślnie!")
println("Liczba warstw: ", length(model.layers))

println("\nArchitektura:")
println("1. Embedding: słowa → wektory $embedding_dim-wymiarowe")
println("2. Conv1D: wykrywa wzory w $kernel_size kolejnych słowach")
println("3. MaxPool: wybiera najważniejsze cechy")
println("4. Flatten: spłaszczenie do wektora")
println("5. Dense: ukryta warstwa z ReLU")
println("6. Dense: klasyfikacja z sigmoid")

Model CNN dla NLP utworzony pomyślnie!
Liczba warstw: 6

Architektura:
1. Embedding: słowa → wektory 3-wymiarowe
2. Conv1D: wykrywa wzory w 2 kolejnych słowach
3. MaxPool: wybiera najważniejsze cechy
4. Flatten: spłaszczenie do wektora
5. Dense: ukryta warstwa z ReLU
6. Dense: klasyfikacja z sigmoid


## 3. Ręczne ustawienie wag dla prostoty obliczeń

Ustawimy proste wartości, które pozwolą na łatwe obliczenia ręczne.

In [6]:
# Embedding weights - każde słowo ma swój unikalny wektor
embedding_weights = Float32[
    1.0  -1.0   0.5  0.0 0.0;   # wymiar 1: <PAD>, good, bad, movie, very
    0.5   0.5   0.0  1.0 0.0;   # wymiar 2
    1.0  -1.0   1.0  0.0 0.0    # wymiar 3
]
model.layers[1].W.output = embedding_weights

println("Embedding weights (3×5 - wymiary × słowa):")
display(embedding_weights)
println("\nInterpretacja embeddingów:")
for (i, word) in enumerate(vocab)
    vec = embedding_weights[:, i]
    println("$word: $vec")
end

3×5 Matrix{Float32}:
 1.0  -1.0  0.5  0.0  0.0
 0.5   0.5  0.0  1.0  0.0
 1.0  -1.0  1.0  0.0  0.0

Embedding weights (3×5 - wymiary × słowa):

Interpretacja embeddingów:
good: Float32[1.0, 0.5, 1.0]
bad: Float32[-1.0, 0.5, -1.0]
movie: Float32[0.5, 0.0, 1.0]
very: Float32[0.0, 1.0, 0.0]
<PAD>: Float32[0.0, 0.0, 0.0]


In [7]:
# Filtry konwolucyjne - wykrywają różne wzory 2-gramów
conv_weights = Float32[
    1.0   0.0;    # Filtr 1: wykrywa pozytywne przejścia
    -1.0  1.0     # Filtr 2: wykrywa negatywne wzory
]
model.layers[2].masks.output = conv_weights

println("Filtry konwolucyjne (kernel_size×num_filters = 2×2):")
display(conv_weights)
println("\nFiltr 1: [1.0, -1.0] - wykrywa pozytywne trendy")
println("Filtr 2: [0.0, 1.0] - wykrywa drugie słowo w parze")

2×2 Matrix{Float32}:
  1.0  0.0
 -1.0  1.0

Filtry konwolucyjne (kernel_size×num_filters = 2×2):

Filtr 1: [1.0, -1.0] - wykrywa pozytywne trendy
Filtr 2: [0.0, 1.0] - wykrywa drugie słowo w parze


In [8]:
# Wagi Dense layers
model.layers[5].W.output = Float32[  # hidden layer weights (4×8)
    1.0   0.5  -0.5  1.0   0.5  -0.5  -0.5  1.0 ;
    0.5   1.0   1.0  0.5   1.0   1.0  1.0  0.5;
    -0.5  0.0   1.0  -0.5  0.0   1.0  1.0  -0.5;
    1.0  -1.0   0.5  1.0  -1.0   0.5  0.5  1.0
]

bias_vector = Float32[0.0; 0.0; 0.0; 0.0]
bias_matrix = reshape(bias_vector, 4, 1)
model.layers[5].b.output = bias_matrix

bias_vector = Float32[0.5 -0.5 1.0 -1.0]
bias_matrix = reshape(bias_vector, 1, 4)
model.layers[6].W.output = bias_matrix

bias_vector = Float32[0.0]
bias_matrix = reshape(bias_vector, 1, 1)
model.layers[6].b.output = bias_matrix

println("Wagi warstwy ukrytej (4×3):")
display(model.layers[5].W.output)
println("\nWagi warstwy wyjściowej (1×4):")
display(model.layers[6].W.output)

4×8 Matrix{Float32}:
  1.0   0.5  -0.5   1.0   0.5  -0.5  -0.5   1.0
  0.5   1.0   1.0   0.5   1.0   1.0   1.0   0.5
 -0.5   0.0   1.0  -0.5   0.0   1.0   1.0  -0.5
  1.0  -1.0   0.5   1.0  -1.0   0.5   0.5   1.0

1×4 Matrix{Float32}:
 0.5  -0.5  1.0  -1.0

Wagi warstwy ukrytej (4×3):

Wagi warstwy wyjściowej (1×4):


## 4. Budowanie grafu obliczeniowego

In [9]:
# Utworzenie węzłów wejściowych
x_input_node = Constant(zeros(Float32, sequence_length, batch_size))
y_label_node = Constant(zeros(Float32, output_size, batch_size))

# Ustaw dane wejściowe
x_input_node.output = X_batch
y_label_node.output = y_batch

println("Dane wejściowe ustawione:")
println("Shape sekwencji X: ", size(x_input_node.output))
println("Shape etykiet y: ", size(y_label_node.output))
println("\nSekwencje jako indeksy:")
display(x_input_node.output)
println("\nEtykiety:")
display(y_label_node.output)

4×2 Matrix{Float32}:
 4.0  4.0
 1.0  2.0
 3.0  3.0
 5.0  5.0

1×2 Matrix{Float32}:
 1.0  0.0

Dane wejściowe ustawione:
Shape sekwencji X: (4, 2)
Shape etykiet y: (1, 2)

Sekwencje jako indeksy:

Etykiety:


In [10]:
# Zbuduj graf obliczeniowy
loss_node, model_output_node, order = build_graph!(model, binarycrossentropy, x_input_node, y_label_node; loss_name="loss")

println("Graf obliczeniowy zbudowany pomyślnie!")
println("Liczba węzłów w grafie: ", length(order))
println("Typ węzła loss: ", typeof(loss_node))
println("Typ węzła output: ", typeof(model_output_node))

Graf obliczeniowy zbudowany pomyślnie!
Liczba węzłów w grafie: 20
Typ węzła loss: ScalarOperator{typeof(Main.MyReverseDiff.binary_cross_entropy_loss_impl)}
Typ węzła output: BroadcastedOperator{typeof(σ)}


## 5. Forward Pass - analiza krok po kroku

Przeanalizujemy jak model przetwarza sekwencje słów.

In [11]:
println("=== DANE WEJŚCIOWE - SEKWENCJE SŁÓW ===")
println("\nSekwencja 1 (pozytywna):")
seq1_words = [vocab[Int(i)] for i in X_batch[:, 1]]
seq1_indices = X_batch[:, 1]
println("Słowa: ", seq1_words)
println("Indeksy: ", seq1_indices)

println("\nSekwencja 2 (negatywna):")
seq2_words = [vocab[Int(i)] for i in X_batch[:, 2]]
seq2_indices = X_batch[:, 2]
println("Słowa: ", seq2_words)
println("Indeksy: ", seq2_indices)

println("\nZadanie: Klasyfikuj sentiment (pozytywny=1, negatywny=0)")

=== DANE WEJŚCIOWE - SEKWENCJE SŁÓW ===

Sekwencja 1 (pozytywna):
Słowa: ["very", "good", "movie", "<PAD>"]
Indeksy: Float32[4.0, 1.0, 3.0, 5.0]

Sekwencja 2 (negatywna):
Słowa: ["very", "bad", "movie", "<PAD>"]
Indeksy: Float32[4.0, 2.0, 3.0, 5.0]

Zadanie: Klasyfikuj sentiment (pozytywny=1, negatywny=0)


In [12]:
# Wykonaj forward pass
println("=== FORWARD PASS ===")
println("Wykonuję forward pass...")
forward_result = forward!(order)
println("Forward pass zakończony pomyślnie!")
println("\nWartość loss: ", loss_node.output)

=== FORWARD PASS ===
Wykonuję forward pass...
Forward pass zakończony pomyślnie!

Wartość loss: 0.7750082


In [13]:
println("=== WYNIKI KLASYFIKACJI SENTIMENTU ===")
println("\nPredykcje modelu: ", model_output_node.output)
println("Prawdziwe etykiety: ", y_label_node.output)
println("Binary Cross Entropy Loss: ", loss_node.output)

# Interpretacja wyników
predictions = model_output_node.output
labels = y_label_node.output

println("\n=== INTERPRETACJA WYNIKÓW ===")
for i in 1:batch_size
    words = [vocab[Int(j)] for j in X_batch[:, i]]
    pred_prob = round(predictions[1, i], digits=4)
    pred_class = pred_prob > 0.5 ? "POZYTYWNY" : "NEGATYWNY"
    true_class = Int(labels[1, i]) == 1 ? "POZYTYWNY" : "NEGATYWNY"
    correct = (pred_prob > 0.5) == (labels[1, i] == 1.0) ? "✓" : "✗"
    
    println("\nSekwencja $i: $(join(words, " "))")
    println("  Predykcja: $pred_prob → $pred_class")
    println("  Prawda: $true_class $correct")
end

=== WYNIKI KLASYFIKACJI SENTIMENTU ===

Predykcje modelu: Float32[0.37754068 0.4378235]
Prawdziwe etykiety: Float32[1.0 0.0]
Binary Cross Entropy Loss: 0.7750082

=== INTERPRETACJA WYNIKÓW ===

Sekwencja 1: very good movie <PAD>
  Predykcja: 0.3775 → NEGATYWNY
  Prawda: POZYTYWNY ✗

Sekwencja 2: very bad movie <PAD>
  Predykcja: 0.4378 → NEGATYWNY
  Prawda: NEGATYWNY ✓


## 6. Weryfikacja ręczna - obliczenia embedding i konwolucji

Policzymy ręcznie pierwsze kroki, żeby zrozumieć jak CNN analizuje tekst.

In [14]:
println("=== WERYFIKACJA RĘCZNA - EMBEDDING ===")
println("\nEmbedding weights:")
display(embedding_weights)

println("\n1. EMBEDDING SEKWENCJI 1: 'very good movie <PAD>'")
println("Indeksy: [5, 2, 4, 1]")

println("\nSlowo 'very' (indeks 5):")
very_embedding = embedding_weights[:, 5]
println("Embedding: ", very_embedding)

println("\nSlowo 'good' (indeks 2):")
good_embedding = embedding_weights[:, 2]
println("Embedding: ", good_embedding)

println("\nSlowo 'movie' (indeks 4):")
movie_embedding = embedding_weights[:, 4]
println("Embedding: ", movie_embedding)

println("\nSlowo '<PAD>' (indeks 1):")
pad_embedding = embedding_weights[:, 1]
println("Embedding: ", pad_embedding)

println("\nEmbedded sekwencja 1 (3×4 - embedding_dim × sequence_length):")
embedded_seq1 = hcat(very_embedding, good_embedding, movie_embedding, pad_embedding)
display(embedded_seq1)

3×5 Matrix{Float32}:
 1.0  -1.0  0.5  0.0  0.0
 0.5   0.5  0.0  1.0  0.0
 1.0  -1.0  1.0  0.0  0.0

3×4 Matrix{Float32}:
 0.0  -1.0  0.0  1.0
 0.0   0.5  1.0  0.5
 0.0  -1.0  0.0  1.0

=== WERYFIKACJA RĘCZNA - EMBEDDING ===

Embedding weights:

1. EMBEDDING SEKWENCJI 1: 'very good movie <PAD>'
Indeksy: [5, 2, 4, 1]

Slowo 'very' (indeks 5):
Embedding: Float32[0.0, 0.0, 0.0]

Slowo 'good' (indeks 2):
Embedding: Float32[-1.0, 0.5, -1.0]

Slowo 'movie' (indeks 4):
Embedding: Float32[0.0, 1.0, 0.0]

Slowo '<PAD>' (indeks 1):
Embedding: Float32[1.0, 0.5, 1.0]

Embedded sekwencja 1 (3×4 - embedding_dim × sequence_length):


In [15]:
println("\n2. KONWOLUCJA 1D - WYKRYWANIE N-GRAMÓW")
println("\nFiltry konwolucyjne (kernel size 2):")
display(conv_weights)
println("Filtr 1: [1.0, -1.0] - wykrywa wzory pozytywne")
println("Filtr 2: [0.0, 1.0] - skupia się na drugim słowie w parze")

println("\nKonwolucja analizuje pary słów:")
println("- Para 1: 'very' + 'good'")
println("- Para 2: 'good' + 'movie'")
println("- Para 3: 'movie' + '<PAD>'")

println("\nPrzykład obliczenia dla pary 'very' + 'good':")
println("Very embedding: ", very_embedding)
println("Good embedding: ", good_embedding)
println("\nFiltr 1 [1.0, -1.0] na wymiarze 1:")
println("1.0 × 0.0 + (-1.0) × 1.0 = -1.0")
println("\nFiltr 2 [0.0, 1.0] na wymiarze 1:")
println("0.0 × 0.0 + 1.0 × 1.0 = 1.0")

println("\nTo pokazuje jak CNN wykrywa różnice między 'very good' a 'very bad'!")

2×2 Matrix{Float32}:
  1.0  0.0
 -1.0  1.0


2. KONWOLUCJA 1D - WYKRYWANIE N-GRAMÓW

Filtry konwolucyjne (kernel size 2):
Filtr 1: [1.0, -1.0] - wykrywa wzory pozytywne
Filtr 2: [0.0, 1.0] - skupia się na drugim słowie w parze

Konwolucja analizuje pary słów:
- Para 1: 'very' + 'good'
- Para 2: 'good' + 'movie'
- Para 3: 'movie' + '<PAD>'

Przykład obliczenia dla pary 'very' + 'good':
Very embedding: Float32[0.0, 0.0, 0.0]
Good embedding: Float32[-1.0, 0.5, -1.0]

Filtr 1 [1.0, -1.0] na wymiarze 1:
1.0 × 0.0 + (-1.0) × 1.0 = -1.0

Filtr 2 [0.0, 1.0] na wymiarze 1:
0.0 × 0.0 + 1.0 × 1.0 = 1.0

To pokazuje jak CNN wykrywa różnice między 'very good' a 'very bad'!


## 7. Test Backward Pass - uczenie się wzorów

In [16]:
println("=== BACKWARD PASS - UCZENIE SIĘ ===")
println("Wykonuję backward pass...")

# Wykonaj backward pass
backward!(order)
println("Backward pass zakończony!")

println("\n=== OBLICZONE GRADIENTY ===")

println("\n1. Gradienty embeddingów (które słowa potrzebują poprawy):")
embedding_grads = model.layers[1].W.gradient
display(embedding_grads)

println("\n2. Gradienty filtrów konwolucyjnych (które wzory poprawić):")
conv_grads = model.layers[2].masks.gradient
display(conv_grads)

println("\n3. Gradienty warstwy klasyfikacyjnej:")
output_grads = model.layers[6].W.gradient
display(output_grads)

=== BACKWARD PASS - UCZENIE SIĘ ===
Wykonuję backward pass...
Backward pass zakończony!

=== OBLICZONE GRADIENTY ===

1. Gradienty embeddingów (które słowa potrzebują poprawy):

2. Gradienty filtrów konwolucyjnych (które wzory poprawić):

3. Gradienty warstwy klasyfikacyjnej:


3×5 Matrix{Float32}:
 0.0         0.0        -0.28815     0.0       0.0
 0.389037   -0.0547279   0.342878    0.334309  0.0
 0.0778074   0.0547279   0.0778074  -0.389037  0.0

2×2 Matrix{Float32}:
 0.711806   0.15133
 0.0547279  0.548936

1×4 Matrix{Float32}:
 0.0632969  -0.507748  -0.340251  -0.155615

In [17]:
a = [0.38903707 -0.054727938; -0.23342223 -0.054727938; 0.07780741 0.05472794; 0.38903707 -0.054727938; -0.23342223 -0.054727938; 0.07780741 0.05472794; 0.07780741 0.05472794; 0.38903707 -0.054727938]
reshape(a,(1,8,2)...)

1×8×2 Array{Float64, 3}:
[:, :, 1] =
 0.389037  -0.233422  0.0778074  0.389037  …  0.0778074  0.0778074  0.389037

[:, :, 2] =
 -0.0547279  -0.0547279  0.0547279  …  0.0547279  0.0547279  -0.0547279

## 8. Podsumowanie i porównanie z Flux

Analiza jak nasza implementacja porównuje się z profesjonalnymi bibliotekami NLP.

In [18]:
println("=== PODSUMOWANIE TESTU CNN DLA NLP ===")

println("\n✓ Zadanie: Klasyfikacja sentimentu tekstu")
println("✓ Dane: 2 sekwencje po 4 słowa")
println("✓ Model: Embedding → Conv1D → Pool → Dense")
println("✓ Forward pass: wykonany")
println("✓ Backward pass: wykonany")
println("✓ Loss: ", round(loss_node.output, digits=6))

println("\n📊 Wyniki klasyfikacji sentimentu:")
for i in 1:batch_size
    words = [vocab[Int(j)] for j in X_batch[:, i]]
    pred = round(model_output_node.output[1, i], digits=4)
    sentiment = pred > 0.5 ? "POZYTYWNY" : "NEGATYWNY"
    true_label = Int(y_label_node.output[1, i]) == 1 ? "POZYTYWNY" : "NEGATYWNY"
    println("  '$(join(words, " "))': $pred → $sentiment (cel: $true_label)")
end

println("\n🔍 Podobieństwa z Flux CNN:")
println("  ✓ Embedding layer - słowa na wektory")
println("  ✓ Conv1D - wykrywanie n-gramów")
println("  ✓ MaxPooling - selekcja najważniejszych cech")
println("  ✓ Dense - klasyfikacja finalna")

println("\n🎯 Zastosowania:")
println("  - Analiza sentimentu (pozytywny/negatywny)")
println("  - Klasyfikacja tematyczna tekstów")
println("  - Wykrywanie spamu")
println("  - Analiza emocji w social media")

println("\n💡 Kluczowe koncepty:")
println("  - N-gramy: CNN wykrywa wzory w parach/trójkach słów")
println("  - Embeddingi: przekształcają słowa w wektory liczb")
println("  - Pooling: wybiera najważniejsze cechy z całego tekstu")
println("  - End-to-end learning: cały model uczy się razem")

=== PODSUMOWANIE TESTU CNN DLA NLP ===

✓ Zadanie: Klasyfikacja sentimentu tekstu
✓ Dane: 2 sekwencje po 4 słowa
✓ Model: Embedding → Conv1D → Pool → Dense
✓ Forward pass: wykonany
✓ Backward pass: wykonany
✓ Loss: 0.775008

📊 Wyniki klasyfikacji sentimentu:
  'very good movie <PAD>': 0.3775 → NEGATYWNY (cel: POZYTYWNY)
  'very bad movie <PAD>': 0.4378 → NEGATYWNY (cel: NEGATYWNY)

🔍 Podobieństwa z Flux CNN:
  ✓ Embedding layer - słowa na wektory
  ✓ Conv1D - wykrywanie n-gramów
  ✓ MaxPooling - selekcja najważniejszych cech
  ✓ Dense - klasyfikacja finalna

🎯 Zastosowania:
  - Analiza sentimentu (pozytywny/negatywny)
  - Klasyfikacja tematyczna tekstów
  - Wykrywanie spamu
  - Analiza emocji w social media

💡 Kluczowe koncepty:
  - N-gramy: CNN wykrywa wzory w parach/trójkach słów
  - Embeddingi: przekształcają słowa w wektory liczb
  - Pooling: wybiera najważniejsze cechy z całego tekstu
  - End-to-end learning: cały model uczy się razem
